In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','amount':'{:,.2f}'
}
cols = 'trade name qty price active'.split()

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 5, 21), datetime.date(2022, 5, 20))

In [2]:
yesterday = date(2021, 5, 20)
yesterday

datetime.date(2021, 5, 20)

In [3]:
format_dict = {
    'qty':'{:,}',
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%',  
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',    
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
}

### Matching check with profits in portmy database with 2022-q1 data (22-05-14 )

In [4]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(59, 18)

In [5]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(36, 23)

In [6]:
df_merge = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge.shape

(59, 41)

In [8]:
names = df_merge[df_merge._merge == 'right_only']['name']
names

Series([], Name: name, dtype: object)

In [9]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

''

In [10]:
sql = """
SELECT Null AS id, name, market, max_price, min_price, daily_volume AS volume, beta 
FROM stocks
WHERE name IN (%s)"""
sql = sql % in_p
my_stocks = pd.read_sql(sql, conmy)
my_stocks

,id,name,market,max_price,min_price,volume,beta


In [11]:
my_stocks.dtypes

id           object
name         object
market       object
max_price    object
min_price    object
volume       object
beta         object
dtype: object

In [12]:
my_stocks['status'] = 'O'
my_stocks['buy_target'] = 0.00
my_stocks['sell_target'] = 0.00
my_stocks['cost'] = 0.00
my_stocks['qty'] = 0
my_stocks['buy_spread'] = -4
my_stocks['sell_spread'] = 4
my_stocks['available_qty'] = 0
my_stocks['bl'] = 0
my_stocks['sh'] = 0
my_stocks['reason'] = 'XXX'
my_stocks.dtypes

id                object
name              object
market            object
max_price         object
min_price         object
volume            object
beta              object
status            object
buy_target       float64
sell_target      float64
cost             float64
qty                int64
buy_spread         int64
sell_spread        int64
available_qty      int64
bl                 int64
sh                 int64
reason            object
dtype: object

In [13]:
rearrange_col = 'id name max_price min_price status buy_target sell_target volume beta cost qty buy_spread sell_spread available_qty bl sh reason market'.split()
rearrange_col

['id',
 'name',
 'max_price',
 'min_price',
 'status',
 'buy_target',
 'sell_target',
 'volume',
 'beta',
 'cost',
 'qty',
 'buy_spread',
 'sell_spread',
 'available_qty',
 'bl',
 'sh',
 'reason',
 'market']

In [14]:
my_stocks = my_stocks[rearrange_col]

In [15]:
rcds = my_stocks.values.tolist()
len(rcds)

0

In [16]:
for rcd in rcds:
    print(rcd)

In [17]:
for rcd in rcds:
    conlite.execute("""INSERT INTO stocks (id, name, max_price, min_price, status, buy_target, sell_target, volume, beta, cost, qty, buy_spread, sell_spread, available_qty, bl, sh, \
reason, market) \
VALUES( ?, ?, ?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?, ?, ?)""", rcd)

### End of Process insert profit records to stocks

### Process for stock records that are not in profits table

In [18]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
stocks = pd.read_sql(sql, conlite)
stocks.shape

(59, 18)

In [19]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
profits = pd.read_sql(sql, conmy)
profits.shape

(36, 23)

In [20]:
df_merge2 = pd.merge(stocks, profits, on='name', how='outer', indicator=True)
df_merge2.shape

(59, 41)

In [21]:
names = df_merge2[df_merge2._merge == 'left_only']['name']
names

2        ASP
4        BGC
6        DCC
7        DIF
10     HREIT
11     JASIF
12     MAKRO
13       MCS
15       NER
17      SCCC
18      SENA
23       TMT
25     WHART
27       BCH
30    DOHOME
34       ORI
39       STA
40     SYNEX
49       KCE
52     PTTGC
53       SCB
54       SCC
58        IP
Name: name, dtype: object

In [22]:
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASP', 'BGC', 'DCC', 'DIF', 'HREIT', 'JASIF', 'MAKRO', 'MCS', 'NER', 'SCCC', 'SENA', 'TMT', 'WHART', 'BCH', 'DOHOME', 'ORI', 'STA', 'SYNEX', 'KCE', 'PTTGC', 'SCB', 'SCC', 'IP'"

In [23]:
sql = '''
SELECT *
FROM stocks
WHERE name IN (%s)
ORDER BY market, name'''
sql = sql % in_p
lite_stocks = pd.read_sql(sql, conlite)
lite_stocks[['market','name']]

,market,name
0,SET,ASP
1,SET,BGC
2,SET,DCC
3,SET,DIF
4,SET,HREIT
5,SET,JASIF
6,SET,MAKRO
7,SET,MCS
8,SET,NER
9,SET,SCCC


### End of Process to delete stock record that are not in profits and avalable_qty = 0

In [24]:
sql = '''
SELECT *
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.dtypes

name                object
q4                 float64
q3                 float64
q2                 float64
q1                 float64
dividend           float64
shares               int64
xdate       datetime64[ns]
paiddate    datetime64[ns]
kind                object
actual               int64
dtype: object

In [25]:
dividend.sort_values(['xdate'],ascending=[True]).style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
5,HTC,1.120000,0.000000,0.830000,0.000000,1.950000,0,2021-09-23 00:00:00,2021-09-23 00:00:00,,0
67,LPF,0.171900,0.189400,0.179300,0.187800,0.728400,0,2022-01-28 00:00:00,2022-02-15 00:00:00,,0
32,GLOBAL,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
70,IP,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
76,DOHOME,0.000000,0.000000,0.000000,0.000000,0.000000,7200,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
79,STARK,0.000000,0.000000,0.000000,0.000000,0.000000,0,2022-02-02 00:00:00,2022-02-02 00:00:00,,0
80,PTTEP,3.000000,0.000000,2.000000,0.000000,5.000000,0,2022-02-10 00:00:00,2022-04-18 00:00:00,,0
77,SCB,2.630000,0.000000,1.430000,0.000000,4.060000,0,2022-02-14 00:00:00,2022-04-22 00:00:00,,0
41,ADVANC,4.240000,0.000000,3.450000,0.000000,7.690000,0,2022-02-18 00:00:00,2022-04-19 00:00:00,,0
44,GVREIT,0.195000,0.176500,0.176500,0.191500,0.739500,0,2022-02-23 00:00:00,2022-09-03 00:00:00,,0


In [26]:
df_merge = pd.merge(stocks, dividend, on='name', how='outer', indicator=True)
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]
#no_dividend.sort_values(['xdate'],ascending=[True])

,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,...,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual,_merge
0,9011.0,AIT,9.80,4.12,O,0.0,0.00,140.03,1.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
5,9709.0,COTTO,3.18,1.55,O,0.0,0.00,10.15,0.87,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
8,9713.0,FORTH,53.00,7.70,O,0.0,0.00,188882.96,0.25,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
9,9188.0,GGC,14.40,9.60,O,0.0,0.00,9.31,0.96,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
19,9449.0,SIS,48.75,25.75,O,30.5,0.00,33.27,1.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
20,9455.0,SMPC,14.20,10.00,O,14.0,0.00,14.45,0.66,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
22,9714.0,SSP,14.27,9.10,O,0.0,0.00,35776.45,0.24,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
28,9053.0,BCPG,15.40,11.10,O,0.0,0.00,157.15,0.95,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
29,9074.0,BPP,20.30,15.30,O,0.0,0.00,28.02,0.83,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only
31,9165.0,ESSO,8.90,7.00,O,0.0,0.00,173.92,1.03,0.00,...,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,left_only


### End of Process

### Consensus table

In [ ]:
sql = '''
SELECT * FROM consensus ORDER BY name'''
css = pd.read_sql(sql, conlite)
css

In [ ]:
sql = '''
SELECT *
FROM profits
ORDER BY name'''
df_prf = pd.read_sql(sql, conmy)
df_prf.shape

In [ ]:
sql = '''
SELECT *
FROM stocks
ORDER BY market, name'''
df_stk = pd.read_sql(sql, conlite)
df_stk.shape

In [ ]:
df_merge = pd.merge(df_prf,df_stk,on='name',how='outer',indicator=True)
df_merge.shape

In [ ]:
left_mask= df_merge._merge == 'left_only'
df_merge[left_mask]

In [ ]:
right_mask= df_merge._merge == 'right_only'
df_merge[right_mask]

In [ ]:
both_mask= df_merge._merge == 'both'
df_merge[both_mask]

In [ ]:
sql = '''
SELECT * FROM SALES ORDER BY name'''
sales = pd.read_sql(sql, conlite)
sales.shape[0]

In [ ]:
sql = '''
DELETE FROM sales
WHERE name = "TCAP"'''
rp = conlite.execute(sql)
rp.rowcount